## 1. Importing Libraries

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

## 2. Reading the Data

In [54]:
PROJECT_DIR = r"C:\Users\ashis\OneDrive\Desktop\flights-sagemaker-project"
DATA_DIR = "data"

In [55]:
# os.path.join(PROJECT_DIR, DATA_DIR)

In [56]:
def get_data(name):
    file_name = f"{name}.csv"
    file_path = os.path.join(PROJECT_DIR, DATA_DIR, file_name)
    return pd.read_csv(file_path)

In [57]:
flights = get_data("flight_price")
flights

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302
...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,9/04/2019,Kolkata,Banglore,CCU → BLR,19:55,22:25,2h 30m,non-stop,No info,4107
10679,Air India,27/04/2019,Kolkata,Banglore,CCU → BLR,20:45,23:20,2h 35m,non-stop,No info,4145
10680,Jet Airways,27/04/2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No info,7229
10681,Vistara,01/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,12648


In [58]:
flights.info()

<class 'pandas.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   Airline          10683 non-null  str  
 1   Date_of_Journey  10683 non-null  str  
 2   Source           10683 non-null  str  
 3   Destination      10683 non-null  str  
 4   Route            10682 non-null  str  
 5   Dep_Time         10683 non-null  str  
 6   Arrival_Time     10683 non-null  str  
 7   Duration         10683 non-null  str  
 8   Total_Stops      10682 non-null  str  
 9   Additional_Info  10683 non-null  str  
 10  Price            10683 non-null  int64
dtypes: int64(1), str(10)
memory usage: 918.2 KB


- The dataset contains 10,683 rows and 11 features
- Columns `Route` and `Total Stops` have missing value each
- The data types of some features isn't appropriate

## 3. Preliminary Analysis

### 3.1 Check Data Types

In [59]:
flights.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [60]:
flights.dtypes

Airline              str
Date_of_Journey      str
Source               str
Destination          str
Route                str
Dep_Time             str
Arrival_Time         str
Duration             str
Total_Stops          str
Additional_Info      str
Price              int64
dtype: object

In [61]:
random_value1 = flights.Date_of_Journey.iloc[6] # checking some random value
random_value1

'12/03/2019'

In [62]:
random_value2 = flights.Dep_Time.iloc[6]
random_value2

'18:55'

### 3.2 Check for Duplicates

In [63]:
flights.duplicated().sum() # Gives number of duplicate rows in our dataset

np.int64(220)

In [64]:
(
    flights
    .loc[flights.duplicated(keep=False)]
    .sort_values(["Airline", "Date_of_Journey", "Source", "Destination" ])
)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6321,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
9848,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
572,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
8168,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
1495,Air India,1/04/2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,01:20 02 Apr,15h 20m,2 stops,No info,10408
...,...,...,...,...,...,...,...,...,...,...,...
2692,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
2870,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
3711,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873
2634,Vistara,24/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,5403


### 3.3 Observations

- The type of `Date_of_Journey`, `Dep_Time` and `Arrival_Time` should be changed to datetime
- The type of `Duration`(2h 50m) and `Total_Stops`(1 stop, 2 stop) is mixed . It should be changed to numeric type (170min).
- There are 220 duplicates. These should be removed.

## 4. Detailed Analysis
- Here we check each feature one by one for accuracy of values.

### Airline

In [65]:
flights.Airline

0             IndiGo
1          Air India
2        Jet Airways
3             IndiGo
4             IndiGo
            ...     
10678       Air Asia
10679      Air India
10680    Jet Airways
10681        Vistara
10682      Air India
Name: Airline, Length: 10683, dtype: str

In [66]:
flights.Airline.unique() # To get all values of a feature

<StringArray>
[                           'IndiGo',                         'Air India',
                       'Jet Airways',                          'SpiceJet',
                 'Multiple carriers',                             'GoAir',
                           'Vistara',                          'Air Asia',
           'Vistara Premium economy',              'Jet Airways Business',
 'Multiple carriers Premium economy',                            'Trujet']
Length: 12, dtype: str

In [67]:
(
    flights
    .Airline
    .str.replace(" Premium economy", "")
    .str.replace(" Business", "")
    .str.title()
    .unique()
)

<StringArray>
[           'Indigo',         'Air India',       'Jet Airways',
          'Spicejet', 'Multiple Carriers',             'Goair',
           'Vistara',          'Air Asia',            'Trujet']
Length: 9, dtype: str

- Some of the entries have inconsistent/inaccurate values (Ex- Vistara & Vistara Premium)

### Date_of_Journey

In [68]:
flights.Date_of_Journey

0        24/03/2019
1         1/05/2019
2         9/06/2019
3        12/05/2019
4        01/03/2019
            ...    
10678     9/04/2019
10679    27/04/2019
10680    27/04/2019
10681    01/03/2019
10682     9/05/2019
Name: Date_of_Journey, Length: 10683, dtype: str

In [69]:
pd.to_datetime(flights.Date_of_Journey, dayfirst=True)

0       2019-03-24
1       2019-05-01
2       2019-06-09
3       2019-05-12
4       2019-03-01
           ...    
10678   2019-04-09
10679   2019-04-27
10680   2019-04-27
10681   2019-03-01
10682   2019-05-09
Name: Date_of_Journey, Length: 10683, dtype: datetime64[us]

### Source

In [70]:
flights.Source.unique()

<StringArray>
['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai']
Length: 5, dtype: str

### Destination

In [71]:
flights.Destination.unique()

<StringArray>
['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad']
Length: 6, dtype: str

### Dep_Time

In [72]:
flights.Dep_Time

0        22:20
1        05:50
2        09:25
3        18:05
4        16:50
         ...  
10678    19:55
10679    20:45
10680    08:20
10681    11:30
10682    10:55
Name: Dep_Time, Length: 10683, dtype: str

In [73]:
(
    flights
    .Dep_Time
    .loc[lambda ser: ser.str.contains("[^0-9:]")] # to confirm whether all values are valid
)

Series([], Name: Dep_Time, dtype: str)

In [74]:
pd.to_datetime(flights.Dep_Time, format="%H:%M").dt.time #.dt.time extracts only the time part from a datetime column.

0        22:20:00
1        05:50:00
2        09:25:00
3        18:05:00
4        16:50:00
           ...   
10678    19:55:00
10679    20:45:00
10680    08:20:00
10681    11:30:00
10682    10:55:00
Name: Dep_Time, Length: 10683, dtype: object

### Arrival_Time

In [75]:
flights.Arrival_Time

0        01:10 22 Mar
1               13:15
2        04:25 10 Jun
3               23:30
4               21:35
             ...     
10678           22:25
10679           23:20
10680           11:20
10681           14:10
10682           19:15
Name: Arrival_Time, Length: 10683, dtype: str

In [76]:
(
    flights
    .Arrival_Time
    .loc[lambda ser: ser.str.contains("[^0-9:]")]
    .str.split(" ", n=1)
    .str.get(1)
    .unique()
)

array(['22 Mar', '10 Jun', '13 Mar', '02 Mar', '10 May', '04 Mar',
       '13 Jun', '28 May', '19 Mar', '07 May', '02 Jun', '16 Jun',
       '19 May', '16 May', '28 Jun', '02 May', '28 Mar', '19 Jun',
       '04 Apr', '25 Mar', '07 Mar', '25 Jun', '07 Jun', '25 May',
       '13 May', '16 Mar', '22 May', '10 Apr', '04 Jun', '20 May',
       '28 Apr', '25 Apr', '10 Mar', '19 Apr', '13 Apr', '02 Apr',
       '23 Mar', '22 Apr', '11 May', '07 Apr', '03 May', '08 Mar',
       '03 Mar', '05 Mar', '22 Jun', '04 May', '26 May', '16 Apr',
       '26 Jun', '29 May', '29 Jun', '29 Mar', '23 May', '17 Jun'],
      dtype=object)

### Duration

In [77]:
flights.Duration

0        2h 50m
1        7h 25m
2           19h
3        5h 25m
4        4h 45m
          ...  
10678    2h 30m
10679    2h 35m
10680        3h
10681    2h 40m
10682    8h 20m
Name: Duration, Length: 10683, dtype: str

In [78]:
(
    flights
    .Duration
    .loc[lambda ser: ~ser.str.contains("m")]
    .unique()
)  

<StringArray>
['19h', '23h', '22h', '12h',  '3h',  '5h', '10h', '18h', '24h', '15h', '16h',
  '8h', '14h', '20h', '13h', '11h',  '9h', '27h', '26h',  '4h',  '7h', '30h',
 '21h', '28h', '47h',  '6h', '25h', '38h', '34h']
Length: 29, dtype: str

In [79]:
(
    flights
    .Duration
    .loc[lambda ser: ~ser.str.contains("h")]
)  

6474    5m
Name: Duration, dtype: str

In [80]:
flights.iloc[[6474]]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6474,Air India,6/03/2019,Mumbai,Hyderabad,BOM → GOI → PNQ → HYD,16:50,16:55,5m,2 stops,No info,17327


- The observation indexed 6474 has duration of 5 minutes which is clearly wrong. Will delete this observation

In [81]:
(
    flights
    .Duration
    .drop(index=6474)
    .str.split(" ", expand=True) #expand=True - each value turns into col of dataframe
    .set_axis(["hour", "minute"], axis=1)
    .assign(
        hour=lambda df_: (
            df_
            .hour
            .str.replace("h", "")
            .astype(int)
            .mul(60)
        ),
        minute=lambda df_: (
            df_
            .minute
            .str.replace("m", "")
            .fillna("0")
            .astype(int)
        )
    )
    # .dtypes
    # .isna().sum()
    .sum(axis=1)
)

0         170
1         445
2        1140
3         325
4         285
         ... 
10678     150
10679     155
10680     180
10681     160
10682     500
Length: 10682, dtype: int64

In [82]:
#For validation

(
    flights
    .Duration
    .drop(index=6474)
    .str.split(" ", expand=True) #expand=True - each value turns into col of dataframe
    .set_axis(["hour", "minute"], axis=1)
    .assign(
        hour=lambda df_: (
            df_
            .hour
            .str.replace("h", "")
            .astype(int)
            .mul(60)
        ),
        minute=lambda df_: (
            df_
            .minute
            .str.replace("m", "")
            .fillna("0")
            .astype(int)
        )
    )
    # .dtypes
    # .isna().sum()
    .sum(axis=1)
    .rename("duration_minutes")
    .to_frame()
    .join(flights.Duration.drop(index=6474))
)

,duration_minutes,Duration
0,170,2h 50m
1,445,7h 25m
2,1140,19h
3,325,5h 25m
4,285,4h 45m
...,...,...
10678,150,2h 30m
10679,155,2h 35m
10680,180,3h
10681,160,2h 40m


### Total Stops

In [83]:
flights.Total_Stops.unique()

<StringArray>
['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops']
Length: 6, dtype: str

In [84]:
(
    flights
    .Total_Stops
    .replace("non-stop", "0")
    .str.replace(" stops?", "", regex=True) #?- means s is optional - finds " stop" or " stops"
    # .astype(int) - can't use this as we have nan here which is float
    .pipe(lambda ser: pd.to_numeric(ser))
)

0        0.0
1        2.0
2        2.0
3        1.0
4        1.0
        ... 
10678    0.0
10679    0.0
10680    0.0
10681    0.0
10682    2.0
Name: Total_Stops, Length: 10683, dtype: float64

### additional_info

In [85]:
flights.Additional_Info.unique()

<StringArray>
[                     'No info',  'In-flight meal not included',
 'No check-in baggage included',              '1 Short layover',
                      'No Info',               '1 Long layover',
              'Change airports',               'Business class',
               'Red-eye flight',               '2 Long layover']
Length: 10, dtype: str

### 5. Cleaning Operations

In [86]:
flights.select_dtypes(include=str).columns # All str columns

Index(['Airline', 'Date_of_Journey', 'Source', 'Destination', 'Route',
       'Dep_Time', 'Arrival_Time', 'Duration', 'Total_Stops',
       'Additional_Info'],
      dtype='str')

In [87]:
def convert_to_minutes(ser):
    return (
        ser
        .str.split(" ", expand=True)
        .set_axis(["hour", "minute"], axis=1)
        .assign(
            hour=lambda df_: (
                df_
                .hour
                .str.replace("h", "")
                .astype(int)
                .mul(60)
            ),
            minute=lambda df_: (
                df_
                .minute
                .str.replace("m", "")
                .fillna("0")
                .astype(int)
            )
            
        )
        # .dtypes
        # .isna().sum()
        .sum(axis=1)
        )

In [88]:
def clean_data(df):
    return (
        df
        .drop(index=[6474])
        .drop_duplicates()
        .assign(**{
            col: df[col].str.strip()
            for col in df.select_dtypes(include=str).columns
        })
        .rename(columns=str.lower)
        .assign(
            airline=lambda df_:(
                df_
                .airline
                .str.replace(" Premium economy", "")
                .str.replace(" Business", "")
                .str.title()
            ),
            date_of_journey=lambda df_:pd.to_datetime(df_.date_of_journey, dayfirst=True),
            dep_time=lambda df_:pd.to_datetime(df_.dep_time).dt.time,
            arrival_time=lambda df_:pd.to_datetime(df_.arrival_time).dt.time,
            duration=lambda df_: df_.duration.pipe(convert_to_minutes), # convert_to_minutes(df_.duration)
            total_stops=lambda df_:  (
                df_
                .total_stops
                .replace("non-stop", "0")
                .str.replace(" stops?", "", regex=True)
                .pipe(lambda ser: pd.to_numeric(ser))
            )
        )
        .drop(columns="route")
    )
    

In [89]:
flights_cleaned = clean_data(flights)#.dtypes #.airline.unique()
flights_cleaned


C:\Users\ashis\AppData\Local\Temp\ipykernel_4356\1105366360.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dep_time=lambda df_:pd.to_datetime(df_.dep_time).dt.time,
C:\Users\ashis\AppData\Local\Temp\ipykernel_4356\1105366360.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  arrival_time=lambda df_:pd.to_datetime(df_.arrival_time).dt.time,


,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-24,Banglore,New Delhi,22:20:00,01:10:00,170,0.0,No info,3897
1,Air India,2019-05-01,Kolkata,Banglore,05:50:00,13:15:00,445,2.0,No info,7662
2,Jet Airways,2019-06-09,Delhi,Cochin,09:25:00,04:25:00,1140,2.0,No info,13882
3,Indigo,2019-05-12,Kolkata,Banglore,18:05:00,23:30:00,325,1.0,No info,6218
4,Indigo,2019-03-01,Banglore,New Delhi,16:50:00,21:35:00,285,1.0,No info,13302
...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,2019-04-09,Kolkata,Banglore,19:55:00,22:25:00,150,0.0,No info,4107
10679,Air India,2019-04-27,Kolkata,Banglore,20:45:00,23:20:00,155,0.0,No info,4145
10680,Jet Airways,2019-04-27,Banglore,Delhi,08:20:00,11:20:00,180,0.0,No info,7229
10681,Vistara,2019-03-01,Banglore,New Delhi,11:30:00,14:10:00,160,0.0,No info,12648


## 6. Split the Data

In [90]:
flights_final = flights_cleaned.sample(1000) #for saving training time (rn)

In [93]:
X = flights_final.drop(columns = "price")
y = flights_final.price.copy()

In [95]:
X_, X_test, y_, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_, y_, test_size=0.2, random_state=42)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(640, 9) (640,)
(160, 9) (160,)
(200, 9) (200,)


## 7. Export the Subsets

In [96]:
def export_data(X, y, name):
    file_name = f'{name}.csv'
    file_path = os.path.join(PROJECT_DIR, DATA_DIR, file_name)

    X.join(y).to_csv(file_path, index=False)

    return pd.read_csv(file_path).head()

In [97]:
export_data(X_train, y_train, "train")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-05-27,Mumbai,Hyderabad,02:35:00,04:05:00,90,0.0,No info,2754
1,Jet Airways,2019-06-09,Delhi,Cochin,14:00:00,12:35:00,1355,1.0,In-flight meal not included,10262
2,Spicejet,2019-05-06,Chennai,Kolkata,08:20:00,10:35:00,135,0.0,No check-in baggage included,3859
3,Air India,2019-05-24,Kolkata,Banglore,14:35:00,14:35:00,1440,1.0,No info,6528
4,Vistara,2019-03-12,Mumbai,Hyderabad,07:30:00,19:55:00,745,1.0,No info,12080


In [98]:
export_data(X_val, y_val, "val")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Spicejet,2019-06-18,Mumbai,Hyderabad,22:45:00,00:15:00,90,0.0,No check-in baggage included,1965
1,Jet Airways,2019-05-27,Delhi,Cochin,14:00:00,12:35:00,1355,1.0,In-flight meal not included,12898
2,Multiple Carriers,2019-06-01,Delhi,Cochin,08:30:00,19:15:00,645,1.0,No info,8937
3,Jet Airways,2019-06-09,Kolkata,Banglore,20:00:00,04:40:00,520,1.0,No info,14178
4,Goair,2019-04-01,Banglore,Delhi,20:55:00,23:40:00,165,0.0,No info,4239


In [99]:
export_data(X_test, y_test, "test")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Vistara,2019-05-18,Kolkata,Banglore,07:10:00,20:20:00,790,1.0,No info,9345
1,Jet Airways,2019-06-21,Mumbai,Hyderabad,08:45:00,10:15:00,90,0.0,No info,8040
2,Indigo,2019-06-06,Kolkata,Banglore,09:20:00,14:15:00,295,1.0,No info,11160
3,Jet Airways,2019-05-01,Kolkata,Banglore,14:05:00,09:20:00,1155,1.0,In-flight meal not included,9663
4,Indigo,2019-06-09,Delhi,Cochin,10:35:00,21:00:00,625,1.0,No info,6314
